In [1]:
import random


In [2]:
import deap
import numpy as np


In [3]:
from deap import base
from deap import creator
from deap import tools


In [4]:
creator.create("FitnessMax", base.Fitness, weights=(1.0,))
creator.create("Individual", list, fitness=creator.FitnessMax)
'''
適合度を最適化（最大化）
個体
'''


'\n適合度を最適化（最大化）\n個体\n'

In [78]:
toolbox = base.Toolbox()
# Attribute generator(個体の作成)
toolbox.register("attr",random.randint,48,96)
# Structure initializers
toolbox.register("individual", tools.initRepeat, creator.Individual, 
    toolbox.attr, 100)#initRepeat　データを入れるコンテナの型、データを生成する関数、コンテナのサイズを引数ロする集団を発生させる関数

toolbox.register("population", tools.initRepeat, list, toolbox.individual)

In [79]:
#案１　音符の落差最小化

def evalOneMax(individual):
    score = 0
    for i in range(0,len(individual)-1):
        if (i >= len(individual)):
            break
        score -= abs(individual[i] - individual[i+1])
    score += sum(individual)
    return score,
'''
この部分を音楽理論に基づいた評価にしたい
方針１：
　制約
 　　１オクターブ以上の音の跳躍を許さない
   　キーにない音を含んだ場合減点
'''


'\nこの部分を音楽理論に基づいた評価にしたい\n方針１：\n\u3000制約\n \u3000\u3000１オクターブ以上の音の跳躍を許さない\n   \u3000キーにない音を含んだ場合減点\n'

In [121]:
#案2　調にない音を減点
def evalOneMax(individual):
    score = 0 
    tonic= 58
    for i in range(0,len(individual)-1):
        if (i >= len(individual)):
            break
        score -= 10*abs(tonic-individual[i])
        if(abs(individual[i]-individual[i+1])>12):
            score -=300
        if(individual[i] % 12 == 0 or 2 or 4 or 7 or 9):
            score += 300
        elif(individual[i] % 12 == 5 or 11):
            score += 100
        else:
            score -= 10000
        
    return score,

In [115]:
toolbox.register("evaluate", evalOneMax)#評価関数
toolbox.register("mate", tools.cxTwoPoints)#交叉
toolbox.register("mutate", tools.mutFlipBit, indpb=0.05)#突然変異
toolbox.register("select", tools.selTournament, tournsize=3)#選択

In [122]:
#遺伝的アルゴリズムによる音列生成
music= list()
def genetic_func(music):
    print ('作成したい曲の調の番号を入力してください。0:\'C\',1:\'C\#\',2:\'D\',3:\'D\#\',4:\'E\',5:\'F\',6:\'F\#\',7:\'G\',8:\'G\#\',9:\'A\',10:\'A\#\',11:\'H\'')
    tonic = input('>>>  ') 
    tonic = int(tonic) + 60
    print(tonic)
    
    random.seed(64)
    # 初期の個体群を生成 N体作成する
    pop = toolbox.population(n=200)
    
    CXPB, MUTPB, NGEN = 0.3, 0.6, 300 # 交差確率、突然変異確率、進化計算のループ回数
    '''
    この部分を、印象値入力にできないだろうか
    '''
    print("Start of evolution")

    # 初期の個体群の評価
    fitnesses = list(map(toolbox.evaluate,pop))
    print(fitnesses)
    print('nyaan')
    for ind, fit in zip(pop, fitnesses):
        ind.fitness.values = fit

    print("  Evaluated %i individuals" % len(pop))

    # 進化計算開始
    for g in range(NGEN):
        print("-- Generation %i --" % g)
        if (g % 30 ==0):
            print("続行しますか？")
            print ('終了する場合は”quit”と入力してください。')
            is_continue = input('>>>  ')
            if (is_continue == 'quit'):
                break
            else:
                continue
                
        # 次世代の個体群
        offspring = toolbox.select(pop, len(pop))
        # 個体群のクローンを生成
        offspring = list(map(toolbox.clone, offspring))

        # 選択した個体群に交差と突然変異を適応する
        # 偶数番目と奇数番目の個体を取り出して交差
        for child1, child2 in zip(offspring[::2], offspring[1::2]):
            if random.random() < CXPB:
                toolbox.mate(child1, child2)
                del child1.fitness.values
                del child2.fitness.values

        for mutant in offspring:
            if random.random() < MUTPB:
                toolbox.mutate(mutant)
                del mutant.fitness.values

        # 適合度が計算されていない個体を集めて適合度を計算
        invalid_ind = [ind for ind in offspring if not ind.fitness.valid]
        fitnesses = map(toolbox.evaluate, invalid_ind)
        for ind, fit in zip(invalid_ind, fitnesses):
            ind.fitness.values = fit

        print("  Evaluated %i individuals" % len(invalid_ind))

        # 次世代群をoffspringにする
        pop[:] = offspring
        
        # すべての個体の適合度を配列にする
        fits = [ind.fitness.values[0] for ind in pop]

        length = len(pop)
        mean = sum(fits) / length
        sum2 = sum(x*x for x in fits)
        std = abs(sum2 / length - mean**2)**0.5

        print("  Min %s" % min(fits))
        print("  Max %s" % max(fits))
        print("  Avg %s" % mean)
        print("  Std %s" % std)

    print("-- End of (successful) evolution --")

    best_ind = tools.selBest(pop, 1)[0]
    print("Best individual is %s, %s" % (best_ind, best_ind.fitness.values))
    music[:] = best_ind[:]
    return music

In [123]:
genetic_func(music)


作成したい曲の調の番号を入力してください。0:'C',1:'C\#',2:'D',3:'D\#',4:'E',5:'F',6:'F\#',7:'G',8:'G\#',9:'A',10:'A\#',11:'H'
>>>  2
62
Start of evolution
[(260,), (-5580,), (-2270,), (-4470,), (-1640,), (-6070,), (-1560,), (-3240,), (-2450,), (-3580,), (-820,), (-4450,), (-6380,), (-2080,), (-1770,), (-1590,), (-810,), (-4270,), (-3170,), (-4640,), (-3700,), (-2360,), (-2570,), (-430,), (-4370,), (-510,), (-3450,), (-5660,), (-3070,), (-3220,), (-4270,), (-4870,), (-2410,), (-1790,), (-2250,), (1020,), (330,), (-1090,), (-660,), (-3060,), (-3920,), (1060,), (-4390,), (-5280,), (-3350,), (-3080,), (-1390,), (-3850,), (-3710,), (-1950,), (-4220,), (-5540,), (-2790,), (-1760,), (-2530,), (-2810,), (-2820,), (-610,), (-4480,), (-4880,), (-2450,), (-1920,), (-3820,), (-2920,), (-1900,), (-2550,), (1880,), (-1120,), (-1550,), (-1670,), (-5730,), (-1770,), (1260,), (-4420,), (-3810,), (-1990,), (-3380,), (30,), (-5730,), (-810,), (-800,), (-3190,), (-4260,), (-5920,), (-2400,), (180,), (-4220,), (-1460,), (-310,

C:\Users\match\Anaconda3\lib\site-packages\deap\tools\crossover.py:61: FutureWarning: tools.cxTwoPoints has been renamed. Use cxTwoPoint instead.
  FutureWarning)



  Evaluated 151 individuals
  Min -10480.0
  Max 3110.0
  Avg -2727.6
  Std 2540.4988171617006
-- Generation 5 --
  Evaluated 150 individuals
  Min -8740.0
  Max 2380.0
  Avg -2776.45
  Std 2456.401412127098
-- Generation 6 --
  Evaluated 157 individuals
  Min -9670.0
  Max 2380.0
  Avg -2758.3
  Std 2498.0990993153173
-- Generation 7 --
  Evaluated 150 individuals
  Min -11230.0
  Max 2340.0
  Avg -3081.75
  Std 2636.5491532493756
-- Generation 8 --
  Evaluated 146 individuals
  Min -9190.0
  Max 4980.0
  Avg -2976.25
  Std 2494.8008813330175
-- Generation 9 --
  Evaluated 135 individuals
  Min -9930.0
  Max 4980.0
  Avg -2620.0
  Std 2692.278774569974
-- Generation 10 --
  Evaluated 146 individuals
  Min -11030.0
  Max 4980.0
  Avg -2395.05
  Std 2840.4448590845764
-- Generation 11 --
  Evaluated 153 individuals
  Min -9000.0
  Max 4980.0
  Avg -2252.85
  Std 2876.2363215667797
-- Generation 12 --
  Evaluated 136 individuals
  Min -12240.0
  Max 4910.0
  Avg -2206.85
  Std 3123.7987

[74,
 68,
 61,
 65,
 57,
 58,
 80,
 83,
 73,
 64,
 0,
 53,
 68,
 57,
 69,
 68,
 92,
 83,
 91,
 81,
 72,
 62,
 55,
 72,
 84,
 80,
 68,
 79,
 75,
 70,
 80,
 89,
 81,
 73,
 75,
 89,
 67,
 63,
 86,
 50,
 53,
 50,
 53,
 92,
 89,
 51,
 51,
 75,
 82,
 85,
 95,
 50,
 93,
 90,
 83,
 75,
 60,
 51,
 92,
 91,
 83,
 64,
 54,
 48,
 59,
 61,
 62,
 62,
 74,
 60,
 69,
 72,
 61,
 54,
 57,
 49,
 50,
 82,
 55,
 48,
 57,
 68,
 70,
 61,
 71,
 67,
 52,
 66,
 58,
 52,
 54,
 71,
 71,
 79,
 58,
 57,
 66,
 67,
 50,
 76]

In [118]:
'''
ニューラルネットワークにより音楽から印象を得る部分
'''
import chainer
import chainer.links as L
import chainer.functions as F

class MLP(chainer.Chain):

    def __init__(self, n_mid_units=40, n_out=4):
        # パラメータを持つ層の登録
        super(MLP, self).__init__(
            l1=L.Linear(None, n_mid_units),
            l2=L.Linear(n_mid_units, n_mid_units),
            l3=L.Linear(n_mid_units, n_out),
        )

    def __call__(self, x):
        # データを受け取った際のforward計算を書く
        h1 = F.relu(self.l1(x))
        h2 = F.relu(self.l2(h1))
        return self.l3(h2)

gpu_id = 0

model = MLP()



In [ ]:
import mido

from mido import Message, MidiFile, MidiTrack

mid = MidiFile()
track = MidiTrack()
mid.tracks.append(track)

track.append(Message('program_change', program=12, time=0))
track.append(Message('note_on', note=64, velocity=64, time=32))
track.append(Message('note_off', note=64, velocity=127, time=32))

mid.save('new_song.mid')

In [18]:
mido.get_output_names()

['Microsoft GS Wavetable Synth 0']

In [19]:
with mido.open_output() as port:
    # 引数が空の場合はデフォルト選択。withにより終了時にポートを閉じる
    for msg in mid.play():
        port.send(msg)

In [26]:
individual?

Object `individual` not found.


In [20]:
#ディープラーニングによる入力に似た曲の生成
import sys
import numpy as np
import chainer
from chainer import cuda, Function, gradient_check, report, training, utils, Variable
from chainer import datasets, iterators, optimizers, serializers
from chainer import Link, Chain, ChainList
import chainer.functions as F
import chainer.links as L
from chainer.training import extensions
import mido 

midi=[]

def get_mid_data(midiname):
    #1パートのみの入力
    mdata=mido.MidiFile('./{0}.mid'.format(midiname))

    for msg in mdata.tracks[1]:
        if msg.type in ['note_on']:
            midi.append(msg.note)
    return mdata

def each_cons(x, size):
    return [x[i:i+size] for i in range(len(x)-size+1)]

class RNN1(Chain):
    def __init__(self):
        super(RNN1, self).__init__(
            embed=L.EmbedID(127, 100),
            mid=L.LSTM(100, 50),  # the first LSTM layer
            out=L.Linear(50, 130),  # the feed-forward output layer
        )

    def reset_state(self):
        self.mid.reset_state()

    def __call__(self, input_vector):
        # Given the current word ID, predict the next word.
        x = self.embed(input_vector)
        h = self.mid(x)
        y = self.out(h)
        return y

rnn = RNN1()
model = L.Classifier(rnn)
optimizer = optimizers.SGD()
optimizer.setup(model)

get_mid_data('BWV772')#いろいろ入力

loss = 0
count = 0
seqlen = len(midi[1:])
for x in range(100):
    print (x)
    rnn.reset_state()
    for cur_event,next_event in each_cons(midi,2):
        cur_event, next_event = np.asarray([cur_event]).astype(np.int32), np.asarray([next_event]).astype(np.int32)
        loss += model(cur_event, next_event)
        count += 1
        if count % 30 == 0 or count == seqlen:
            #print loss.data
            model.zerograds()
            loss.backward()
            loss.unchain_backward()
            optimizer.update()


model.predictor.reset_state()
l = list()
prev_event = chainer.Variable(np.array([64], np.int32))
prob = F.softmax(model.predictor(prev_event))
l.append(59)

for i in range(300):
    prob = F.softmax(model.predictor(prev_event))
    probability = prob.data[0].astype(np.float64)
    probability /= np.sum(probability)
    index = np.random.choice(range(len(probability)), p=probability)

    l.append(index)

    prev_event = chainer.Variable(np.array([index], dtype=np.int32))

print (l)
print (len(l))

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
[59, 60, 62, 64, 65, 62, 64, 74, 67, 72, 71, 69, 67, 71, 74, 69, 71, 72, 74, 71, 69, 72, 71, 74, 72, 76, 74, 72, 71, 69, 69, 71, 67, 74, 72, 71, 69, 67, 74, 66, 69, 71, 72, 74, 71, 69, 71, 74, 72, 81, 80, 83, 79, 77, 76, 74, 72, 68, 69, 81, 77, 76, 74, 72, 74, 72, 71, 69, 69, 81, 41, 65, 79, 120, 43, 69, 71, 62, 64, 65, 62, 64, 65, 62, 64, 60, 67, 72, 71, 69, 71, 67, 74, 72, 69, 71, 74, 72, 69, 71, 69, 62, 64, 67, 66, 69, 71, 72, 74, 71, 72, 74, 71, 69, 66, 68, 64, 76, 74, 72, 76, 74, 73, 74, 76, 77, 69, 71, 74, 72, 71, 74, 73, 76, 74, 73, 74, 76, 77, 81, 79, 77, 76, 74, 77, 76, 74, 72, 74, 76, 77, 79, 81, 83, 84, 81, 83, 79, 84, 122, 79, 76, 77, 76, 74, 72, 74, 76, 77, 74, 76, 72, 81, 80, 83, 81, 76,

In [49]:
import pretty_midi as pm
#LSTMが出力したノート番号の配列を元にMIDI楽曲ファイルを作る


def makemidi(mlist,delta,mname):
    song = pm.PrettyMIDI()             #楽曲ファイルを生成
    piano = pm.Instrument(program=1)   #音色にピアノをセット
    s=0                                #時間の初期化
    
    for i in range(len(mlist)):
        if mlist[i] > 127:   #ノート番号が１２７を超えた場合休符にする
            mlist[i]=0
        
        note=pm.Note(velocity =64, pitch = mlist[i], start =s, end =s+delta/2) 
    
        '''
        音符の長さも変えたい
        '''
        s =s+delta/2
        piano.notes.append(note)

    
    song.instruments.append(piano)

    song.write('{0}.mid'.format(mname))#song.midに書き出し
    print('Success to make {0}.mid !!\n'.format(mname))

In [15]:
music?

In [109]:
import pandas as pd
import mido

mid = mido.MidiFile('./ga3.mid')
scale = ('C', 'C#', 'D', 'D#', 'E', 'F', 'F#', 'G', 'G#', 'A', 'A#', 'H')
on_notes = dict()
delta = 240
tmp_score = []
mid.print_tracks()

for track in mid.tracks:
    e_time = 0
    for msg in track:
        e_time += msg.time
        if msg.type == 'note_on':
            if msg.note in on_notes: raise
            on_notes[msg.note] = e_time
        elif msg.type == 'note_off':
            if not msg.note in on_notes: raise
            on_time = on_notes.pop(msg.note)
            dur = (e_time - on_time) / delta
            tmp_score.append([msg.note, 
                              scale[msg.note%12], 
                              msg.note//12, 
                              dur, 
                              msg.velocity, 
                              msg.channel, 
                              on_time / delta])

score = pd.DataFrame(tmp_score, columns=['noteID', 'note','octave','dur','vel','ch','time'])
score

=== Track 0
<meta message time_signature numerator=4 denominator=4 clocks_per_click=24 notated_32nd_notes_per_beat=8 time=0>
<meta message set_tempo tempo=500000 time=0>
<meta message end_of_track time=1>
=== Track 1
<message program_change channel=0 program=1 time=0>
<meta message end_of_track time=1>


,noteID,note,octave,dur,vel,ch,time


In [124]:
makemidi(music,0.35,'ga3')

Success to make ga3.mid !!



In [27]:
makemidi(l,0.35,'invention772')

Success to make invention772.mid !!



In [144]:
i?

In [33]:
print ('何か文字を入力してください。')
test = input('>>>  ')
if (test=='hello,world'):
    print(test)

何か文字を入力してください。
>>>  hello,world
hello,world


In [86]:
score= 100
individual = list()
individual.append(3)
individual.append(4)
individual.append(9)
score += abs(individual[1] - individual[2]) 
print(score)

105
